In [1]:
!pip freeze | grep scikit-learn
#sci-kit learn version

scikit-learn==1.5.0


In [2]:
!python -V

Python 3.9.19


In [3]:
import pickle
import pandas as pd
import os

In [4]:
#loading pickle file
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
#location id and drop off location id are categorical features
categorical = ['PULocationID', 'DOLocationID']

#remove outliers
#fill blank values
def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
#download yellow march 2023 taxi data
df = read_data('./data/yellow_tripdata_2023-03.parquet')

In [7]:
#categorical features turn into dictionary 
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [8]:
#Q1: the standard deviation of the predicted duration for March 2023 yellow taxi data
y_pred.std()

6.247488852238703

In [9]:
#Q2. Preparing the output
year=2023
month=3
output_file = f'output/yellow_tripprediction_{year:04d}-{month:02d}.parquet'
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

os.makedirs('output', exist_ok=True)
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

#What's the size of the output file?
!ls -lh output/

total 66M
-rw-rw-r-- 1 nevin nevin 66M Jun 17 14:59 yellow_tripprediction_2023-03.parquet


In [10]:
#Q3. Creating the scoring script
!jupyter nbconvert --to script hw4.ipynb

[NbConvertApp] Converting notebook hw4.ipynb to script
[NbConvertApp] Writing 2359 bytes to hw4.py


In [11]:
#Q4. Virtual environment
!pipenv install scikit-learn==1.5.0 pandas pyarrow

Creating a Pipfile for this project...
Installing scikit-learn==1.5.0...
Resolving scikit-learn==1.5.0...
Added scikit-learn to Pipfile's [packages] ...
✔ Installation Succeeded-learn...
⠋ Installing scikit-learn...
Installing pandas...
Resolving pandas...
Added pandas to Pipfile's [packages] ...
✔ Installation Succeeded...
⠋ Installing pandas...
Installing pyarrow...
Resolving pyarrow...
Added pyarrow to Pipfile's [packages] ...
✔ Installation Succeededw...
⠋ Installing pyarrow...
Pipfile.lock not found, creating...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking packages...
⠇ Locking packages...
Locking [dev-packages] dependencies...
Updated Pipfile.lock (2685029e49c649716e23433a663529ece591cc78592a9ccdcadd992d6786940c)!
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing dependencies from Pipfile.lock (86940c)...


First scikit-learn hash: "sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c"

In [12]:
#Q5. Parametrize the script
!python hw4.py 2023 4

predicted mean duration for year 2023 month 4 is: 14.292282936862449


In [13]:
#Q6. Docker container
!docker build --tag trip_predict_app .

[+] Building 0.0s (0/0)  docker:default
[+] Building 0.0s (0/0)  docker:default
[+] Building 0.0s (0/0)  docker:default
[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (0/1)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.2s
 => => transferring dockerfile: 281B                                       0.0s
[+] Building 0.2s (1/1)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.2s
 => => transferring dockerfile: 281B                                       0.0s
[+] Building 0.5s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.2s
 => => transferring dockerfile: 281B                                       0.0s
 => [internal] load metadata for docker.io/agrigorev/zoomcamp-model:mlops  0.2s


In [14]:
!docker run --rm trip_predict_app 2023 5

predicted mean duration for year 2023 month 5 is: 0.19174419265916945
